In [1]:
import numpy as np
import scipy
from scipy.optimize import minimize
from scipy.linalg import cho_factor, cho_solve

In [2]:
def params_unwrap(param_vec, shapes, sizes):
    """Helper routine for minimize_list"""
    args = []
    pos = 0
    for i in range(len(shapes)):
        sz = sizes[i]
        args.append(param_vec[pos:pos+sz].reshape(shapes[i]))
        pos += sz
    return args


def params_wrap(param_list):
    """Helper routine for minimize_list"""
    param_list = [np.array(x) for x in param_list]
    shapes = [x.shape for x in param_list]
    sizes = [x.size for x in param_list]
    param_vec = np.zeros(sum(sizes))
    pos = 0
    for param in param_list:
        sz = param.size
        param_vec[pos:pos+sz] = param.ravel()
        pos += sz
    unwrap = lambda pvec: params_unwrap(pvec, shapes, sizes)
    return param_vec, unwrap


def minimize_list(cost, init_list, args):
    """Optimize a list of arrays (wrapper of scipy.optimize.minimize)

    The input function "cost" should take a list of parameters,
    followed by any extra arguments:
        cost(init_list, *args)
    should return the cost of the initial condition, and a list in the same
    format as init_list giving gradients of the cost wrt the parameters.

    The options to the optimizer have been hard-coded. You may wish
    to change disp to True to get more diagnostics. You may want to
    decrease maxiter while debugging. Although please report all results
    in Q2-5 using maxiter=500.
    """
    opt = {'maxiter': 500, 'disp': False}
    init, unwrap = params_wrap(init_list)
    def wrap_cost(vec, *args):
        E, params_bar = cost(unwrap(vec), *args)
        vec_bar, _ = params_wrap(params_bar)
        return E, vec_bar
    res = minimize(wrap_cost, init, args, 'L-BFGS-B', jac=True, options=opt)
    return unwrap(res.x)


def linreg_cost(params, X, yy, alpha):
    """Regularized least squares cost function and gradients

    Can be optimized with minimize_list -- see fit_linreg_gradopt for a
    demonstration.

    Inputs:
    params: tuple (ww, bb): weights ww (D,), bias bb scalar
         X: N,D design matrix of input features
        yy: N,  real-valued targets
     alpha: regularization constant

    Outputs: (E, [ww_bar, bb_bar]), cost and gradients
    """
    # Unpack parameters from list
    ww, bb = params

    # forward computation of error
    ff = np.dot(X, ww) + bb
    res = ff - yy
    E = np.dot(res, res) + alpha*np.dot(ww, ww)

    # reverse computation of gradients
    ff_bar = 2*res
    bb_bar = np.sum(ff_bar)
    ww_bar = np.dot(X.T, ff_bar) + 2*alpha*ww

    return E, [ww_bar, bb_bar]


def fit_linreg_gradopt(X, yy, alpha):
    """
    fit a regularized linear regression model with gradient opt

         ww, bb = fit_linreg_gradopt(X, yy, alpha)

     Find weights and bias by using a gradient-based optimizer
     (minimize_list) to improve the regularized least squares cost:

       np.sum(((np.dot(X,ww) + bb) - yy)**2) + alpha*np.dot(ww,ww)

     Inputs:
             X N,D design matrix of input features
            yy N,  real-valued targets
         alpha     scalar regularization constant

     Outputs:
            ww D,  fitted weights
            bb     scalar fitted bias
    """
    D = X.shape[1]
    args = (X, yy, alpha)
    init = (np.zeros(D), np.array(0))
    ww, bb = minimize_list(linreg_cost, init, args)
    return ww, bb


def logreg_cost(params, X, yy, alpha):
    """Regularized logistic regression cost function and gradients

    Can be optimized with minimize_list -- see fit_linreg_gradopt for a
    demonstration of fitting a similar function.

    Inputs:
    params: tuple (ww, bb): weights ww (D,), bias bb scalar
         X: N,D design matrix of input features
        yy: N,  real-valued targets
     alpha: regularization constant

    Outputs: (E, [ww_bar, bb_bar]), cost and gradients
    """
    # Unpack parameters from list
    ww, bb = params

    # Force targets to be +/- 1
    yy = 2*(yy==1) - 1

    # forward computation of error
    aa = yy*(np.dot(X, ww) + bb)
    sigma = 1/(1 + np.exp(-aa))
    E = -np.sum(np.log(sigma)) + alpha*np.dot(ww, ww)

    # reverse computation of gradients
    aa_bar = sigma - 1
    bb_bar = np.dot(aa_bar, yy)
    ww_bar = np.dot(X.T, yy*aa_bar) + 2*alpha*ww

    return E, (ww_bar, bb_bar)


def nn_cost(params, X, yy=None, alpha=None):
    """NN_COST simple neural network cost function and gradients, or predictions

           E, params_bar = nn_cost([ww, bb, V, bk], X, yy, alpha)
                    pred = nn_cost([ww, bb, V, bk], X)

     Cost function E can be minimized with minimize_list

     Inputs:
             params (ww, bb, V, bk), where:
                    --------------------------------
                        ww K,  hidden-output weights
                        bb     scalar output bias
                        V  K,D hidden-input weights
                        bk K,  hidden biases
                    --------------------------------
                  X N,D input design matrix
                 yy N,  regression targets
              alpha     scalar regularization for weights

     Outputs:
                     E  sum of squares error
            params_bar  gradients wrt params, same format as params
     OR
               pred N,  predictions if only params and X are given as inputs
    """
    # Unpack parameters from list
    ww, bb, V, bk = params

    # Forwards computation of cost
    A = np.dot(X, V.T) + bk[None,:] # N,K
    P = 1 / (1 + np.exp(-A)) # N,K
    F = np.dot(P, ww) + bb # N,
    if yy is None:
        # user wants prediction rather than training signal:
        return F
    res = F - yy # N,
    E = np.dot(res, res) + alpha*(np.sum(V*V) + np.dot(ww,ww)) # 1x1

    # Reverse computation of gradients
    F_bar = 2*res # N,
    ww_bar = np.dot(P.T, F_bar) + 2*alpha*ww # K,
    bb_bar = np.sum(F_bar) # scalar
    P_bar = np.dot(F_bar[:,None], ww[None,:]) # N,K
    A_bar = P_bar * P * (1 - P) # N,K
    V_bar = np.dot(A_bar.T, X) + 2*alpha*V # K,D
    bk_bar = np.sum(A_bar, 0)

    return E, (ww_bar, bb_bar, V_bar, bk_bar)


def rbf_fn(X1, X2):
    """Helper routine for gp_post_par"""
    return np.exp((np.dot(X1,(2*X2.T))-np.sum(X1*X1,1)[:,None]) - np.sum(X2*X2,1)[None,:])


def gauss_kernel_fn(X1, X2, ell, sigma_f):
    """Helper routine for gp_post_par"""
    return sigma_f**2 * rbf_fn(X1/(np.sqrt(2)*ell), X2/(np.sqrt(2)*ell))


def gp_post_par(X_rest, X_obs, yy, sigma_y=0.05, ell=5.0, sigma_f=0.1):
    """GP_POST_PAR means and covariances of a posterior Gaussian process

         rest_cond_mu, rest_cond_cov = gp_post_par(X_rest, X_obs, yy)
         rest_cond_mu, rest_cond_cov = gp_post_par(X_rest, X_obs, yy, sigma_y, ell, sigma_f)

     Calculate the means and covariances at all test locations of the posterior Gaussian
     process conditioned on the observations yy at observed locations X_obs.

     Inputs:
                 X_rest GP test locations
                  X_obs locations of observations
                     yy observed values
                sigma_y observation noise standard deviation
                    ell kernel function length scale
                sigma_f kernel function standard deviation

     Outputs:
           rest_cond_mu mean at each location in X_rest
          rest_cond_cov covariance matrix between function values at all test locations
    """
    X_rest = X_rest[:, None]
    X_obs = X_obs[:, None]
    K_rest = gauss_kernel_fn(X_rest, X_rest, ell, sigma_f)
    K_rest_obs = gauss_kernel_fn(X_rest, X_obs, ell, sigma_f)
    K_obs = gauss_kernel_fn(X_obs, X_obs, ell, sigma_f)
    M = K_obs + sigma_y**2 * np.eye(yy.size)
    M_cho, M_low = cho_factor(M)
    rest_cond_mu = np.dot(K_rest_obs, cho_solve((M_cho, M_low), yy))
    rest_cond_cov = K_rest - np.dot(K_rest_obs, cho_solve((M_cho, M_low), K_rest_obs.T))

    return rest_cond_mu, rest_cond_cov


In [3]:
data = np.load('./ct_data.npz')
X_train = data['X_train']; X_val = data['X_val']; X_test = data['X_test']
y_train = data['y_train']; y_val = data['y_val']; y_test = data['y_test']

# Q1

## a)

In [4]:
print(f'The mean of the training positions in y_train is {np.mean(y_train):.4f} and the standard error of the mean is {np.std(y_train)/np.sqrt(len(y_train)):.4f}.')
print(f'The mean of the 5,785 positions in the y_val is {np.mean(y_val):.4f} and the standard error of the mean is  {np.std(y_val)/np.sqrt(len(y_val)):.4f}.')
print(f'The mean of the first 5,785 entries in the y_train is {np.mean(y_train[0:5785,]):.4f} and the standard error of the mean is  {np.std(y_train[0:5785,])/np.sqrt(5785):.4f}.')

The mean of the training positions in y_train is -0.0000 and the standard error of the mean is 0.0050.
The mean of the 5,785 positions in the y_val is -0.2160 and the standard error of the mean is  0.0129.
The mean of the first 5,785 entries in the y_train is -0.4425 and the standard error of the mean is  0.0119.


## b)

In [5]:
# stage one: constant columns
index1 = np.where(np.var(X_train, axis=0) == 0)[0]
print(f'Column indexes of constants: {index1}.')

# stage two: identical columns
index2 = np.delete(np.arange(0, X_train.shape[1]), np.sort(np.unique(X_train, axis=1, return_index=True)[1]))
print(f'Column indexes of dupulicates: {index2}.')

# indeices for removal
index_remove = np.unique(np.concatenate((index1, index2), axis=0))
print(f'Column indexes for removal: {index_remove}.')

# removal
X_train = np.delete(X_train, index_remove, axis=1)
X_val = np.delete(X_val, index_remove, axis=1)
X_test = np.delete(X_test, index_remove, axis=1)

Column indexes of constants: [ 59  69 179 189 351].
Column indexes of dupulicates: [ 69  78  79 179 188 189 199 287 351 359].
Column indexes for removal: [ 59  69  78  79 179 188 189 199 287 351 359].


# Q2

In [6]:
def fit_linreg(X, yy, alpha):
    n, d = X.shape[0], X.shape[1]
    y_tilde = np.concatenate((yy, np.zeros(d)))
    Phi = np.concatenate((X, np.ones((n, 1))), axis=1)
    diag = np.diag(np.append(np.sqrt(alpha) * np.ones((d)), [0]))
    Phi_tilde = np.concatenate((Phi, diag))
    y_tilde = np.concatenate((yy, np.zeros((d+1))))
    w_tilde = np.linalg.lstsq(Phi_tilde, y_tilde, rcond=None)[0]
    ww, bb = w_tilde[0:-1], w_tilde[-1]
    return ww, bb

In [7]:
# fit X_train and y_train with alpha = 30 using least square method
w_ls, b_ls = fit_linreg(X=X_train, yy=y_train, alpha=30)
pred_ls = np.concatenate((X_train, np.ones((len(X_train), 1))), axis=1) @ np.concatenate((w_ls, b_ls), axis=None)

In [8]:
# fit X_train and y_train with alpha = 30 using gradient descent method
w_gd, b_gd = fit_linreg_gradopt(X=X_train, yy=y_train, alpha=30)
pred_gd = np.concatenate((X_train, np.ones((len(X_train), 1))), axis=1) @ np.concatenate((w_ls, b_ls), axis=None)

In [9]:
RMSE_ls = np.sqrt(np.mean((y_train - pred_ls) ** 2))
RMSE_gd = np.sqrt(np.mean((y_train - pred_gd) ** 2))
print(f'The RMSE using least square method is {RMSE_ls:.4f}.')
print(f'The RMSE using gradient descent method is {RMSE_gd:.4f}.')

The RMSE using least square method is 0.3568.
The RMSE using gradient descent method is 0.3568.


# Q3

In [10]:
def fit_logreg_gradopt(X, yy, alpha):
    D = X.shape[1]
    args = (X, yy, alpha)
    init = (np.zeros(D), np.array([0]))
    ww, bb = minimize_list(logreg_cost, init, args)
    return ww, bb

def logreg_prob(X, ww, bb):
    return 1 / (1 + np.exp(- (X @ ww + bb)))

In [11]:
K = 20 # number of thresholded classification problems to fit
mx = np.max(y_train); mn = np.min(y_train); hh = (mx-mn)/(K+1)
thresholds = np.linspace(mn+hh, mx-hh, num=K, endpoint=True)
prob_train = np.zeros((len(X_train), K))
prob_val = np.zeros((len(X_val), K))
w_logr = np.zeros((X_train.shape[1], K))
b_logr = np.zeros(K)

for kk in range(K):
    labels = (y_train > thresholds[kk]).astype(int)
    w_logr_kk, b_logr_kk = fit_logreg_gradopt(X=X_train, yy=labels, alpha=30)
    w_logr[:,kk] = w_logr_kk
    b_logr[kk] = b_logr_kk[0]
    prob_train[:,kk] = logreg_prob(X=X_train, ww=w_logr[:,kk], bb=b_logr[kk])
    prob_val[:,kk] = logreg_prob(X=X_val, ww=w_logr[:,kk], bb=b_logr[kk])

In [12]:
w_logr_lr, b_logr_lr = fit_linreg_gradopt(X=prob_train, yy=y_train, alpha=30)
param_logr_lr = np.concatenate((w_logr_lr, b_logr_lr), axis=None)

In [13]:
pred_logr_lr_train = np.concatenate((prob_train, np.ones((len(prob_train), 1))), axis=1) @ param_logr_lr
RMSE_log_lr_train = np.sqrt(np.mean((y_train - pred_logr_lr_train) ** 2))
print(f'The RMSE of the TRAIN set is {RMSE_log_lr_train:.4f}.')

The RMSE of the TRAIN set is 0.1544.


In [14]:
pred_log_lr_val = np.concatenate((prob_val, np.ones((len(prob_val), 1))), axis=1) @ param_logr_lr
RMSE_log_lr_val = np.sqrt(np.mean((y_val - pred_log_lr_val) ** 2))
print(f'The RMSE of the VAL set is {RMSE_log_lr_val:.4f}.')

The RMSE of the VAL set is 0.2542.


# Q4

In [15]:
# random initialisation
d, k = X_train.shape[1], 20
ww = 0.1*np.random.randn(k)/np.sqrt(k)
bb = 0.1*np.random.randn()/np.sqrt(k)
V = 0.1*np.random.randn(k, d)/np.sqrt(k)
bk = 0.1*np.random.randn(k)/np.sqrt(k)
nn_init_random = (ww, bb, V, bk)

# X_train, y_train and alpha
nn_alpha = 30
nn_args = (X_train, y_train, nn_alpha)

nn_param_random = minimize_list(nn_cost, nn_init_random, nn_args)

pred_nn_random_train = nn_cost(nn_param_random, X=X_train)
RMSE_nn_random_train = np.sqrt(np.mean((y_train - pred_nn_random_train) ** 2))

print(f'The RMSE of nn with random initialisaiton using the TRAIN set is {RMSE_nn_random_train:.4f}.')

The RMSE of nn with random initialisaiton using the TRAIN set is 0.1411.


In [16]:
# Q3 initialisation
nn_init_q3 = (w_logr_lr, b_logr_lr, w_logr.T, b_logr)

# X_train, y_train and alpha
nn_alpha = 30
nn_args = (X_train, y_train, nn_alpha)

nn_params_q3 = minimize_list(nn_cost, nn_init_q3, nn_args)

pred_nn_q3_train = nn_cost(nn_params_q3, X=X_train)
RMSE_nn_q3_train = np.sqrt(np.mean((y_train - pred_nn_q3_train) ** 2))

print(f'The RMSE of nn with Q3 initialisaiton using the TRAIN set is {RMSE_nn_q3_train:.4f}.')

The RMSE of nn with Q3 initialisaiton using the TRAIN set is 0.1394.


# Q5

In [17]:
def train_nn_reg(XX, XX_val, yy, yy_val, params, alpha, report_train=False, report_val=True):
    args = (XX, yy, alpha)
    params_bar = minimize_list(nn_cost, params, args)
    pred_train = nn_cost(params_bar, X=XX)
    RMSE_train = np.sqrt(np.mean((yy - pred_train) ** 2))
    pred_val = nn_cost(params_bar, X=XX_val)
    RMSE_val = np.sqrt(np.mean((yy_val - pred_val) ** 2))
    if report_train:
        print(f'The RMSE_train is {RMSE_train:.4f} while alpha is {alpha:.2f}.')
    if report_val:
        print(f'The RMSE_val is {RMSE_val:.4f} while alpha is {alpha:.2f}.')
    return RMSE_val

In [21]:
# alpha set
alpha_set = np.arange(0, 50, 0.02)
# Q3 initialisation
nn_init_q3 = (w_logr_lr, b_logr_lr, w_logr.T, b_logr)
points_num = 3
alpha_init = 30
RMSE_alpha_base = train_nn_reg(XX=X_train, XX_val=X_val, yy=y_train, yy_val=y_val, params=nn_init_q3, alpha=alpha_init)
print('The above RMSE is a baseline. \n')
best_RMSE_val = RMSE_alpha_base
best_alpha = alpha_init

for i in range(5):
    pick = np.random.randint(0, len(alpha_set), points_num)
    alpha_train = alpha_set[pick]
    alpha_acquisition = np.delete(alpha_set, pick)
    alpha_set = np.delete(alpha_set, pick)

    print(f'---------- Iteration {i+1} ----------')
    for j in range(points_num):
        RMSE_found = np.append(RMSE_found, train_nn_reg(XX=X_train, XX_val=X_val, yy=y_train, yy_val=y_val, params=nn_init_q3, alpha=alpha_train[j]))
        alpha_found = np.append(alpha_found, alpha_train[j])
        
    y = np.log(RMSE_alpha_base) - np.log(RMSE_found)
    # mu, cov = gp_post_par(X_rest=alpha_acquisition, X_obs=alpha_train, yy=y)
    mu, cov = gp_post_par(X_rest=alpha_acquisition, X_obs=alpha_found, yy=y)
    pi = scipy.stats.norm.cdf((mu - y.max()) / np.sqrt(np.diag(cov)))
    alpha_opt = alpha_acquisition[np.argmax(pi)]
    print(f'The maximal PI is {pi.max():.4f} while alpha equals to {alpha_opt:.2f}.')

    alpha_set = np.delete(alpha_set, np.where(alpha_set == alpha_opt))
    RMSE_found = np.append(RMSE_found, train_nn_reg(XX=X_train, XX_val=X_val, yy=y_train, yy_val=y_val, params=nn_init_q3, alpha=alpha_opt))
    alpha_found = np.append(alpha_found, alpha_opt)

    print('\n')

best_RMSE_val = RMSE_found.min()
best_alpha = alpha_found[np.where(RMSE_found == RMSE_found.min())][0]
print(f'The best RMSE of the VAL set is {best_RMSE_val:.4f} while alpha is {best_alpha:.2f}.')

The RMSE_val is 0.2703 while alpha is 30.00.
The above RMSE is a baseline. 

---------- Iteration 1 ----------
The RMSE_val is 0.2734 while alpha is 37.24.
The RMSE_val is 0.2486 while alpha is 3.80.
The RMSE_val is 0.2733 while alpha is 37.84.
The maximal PI is 0.3537 while alpha equals to 3.78.
The RMSE_val is 0.2573 while alpha is 3.78.


---------- Iteration 2 ----------
The RMSE_val is 0.2800 while alpha is 46.70.
The RMSE_val is 0.2561 while alpha is 13.08.
The RMSE_val is 0.2719 while alpha is 32.56.
The maximal PI is 0.3414 while alpha equals to 7.62.
The RMSE_val is 0.2451 while alpha is 7.62.


---------- Iteration 3 ----------
The RMSE_val is 0.2709 while alpha is 31.04.
The RMSE_val is 0.2436 while alpha is 0.32.
The RMSE_val is 0.2663 while alpha is 18.10.
The maximal PI is 0.2588 while alpha equals to 0.00.
The RMSE_val is 0.2576 while alpha is 0.00.


---------- Iteration 4 ----------
The RMSE_val is 0.2598 while alpha is 15.58.
The RMSE_val is 0.2718 while alpha is 31.7

In [ ]:
# # alpha set
# alpha_set = np.arange(0, 50, 0.02)
# # Q3 initialisation
# nn_init_q3 = (w_logr_lr, b_logr_lr, w_logr.T, b_logr)
# points_num = 3
# RMSE_alpha_base = train_nn_reg(XX=X_train, XX_val=X_val, yy=y_train, yy_val=y_val, params=nn_init_q3, alpha=30)
# print('The above RMSE is a baseline.')

# for i in range(5):
#     pick = np.random.randint(0, len(alpha_set), points_num)
#     alpha_train = alpha_set[pick]
#     alpha_acquisition = np.delete(alpha_set, pick)
#     RMSE_val = np.zeros(points_num)
#     for j in range(points_num):
#         RMSE_val[j] = train_nn_reg(XX=X_train, XX_val=X_val, yy=y_train, yy_val=y_val, params=nn_init_q3, alpha=alpha_train[j])
#     y = np.log(RMSE_alpha_base) - np.log(RMSE_val)
    
#     mu, cov = gp_post_par(X_rest=alpha_acquisition, X_obs=alpha_train, yy=y)
#     pi = scipy.stats.norm.cdf((mu - y.max()) / np.sqrt(np.diag(cov)))
#     alpha_opt = alpha_acquisition[np.argmax(pi)]
#     print(f'Iteration {i+1}, the maximal PI is {pi.max()} while alpha equals to {alpha_opt:.2f}.')
#     RMSE_alpha_base = train_nn_reg(XX=X_train, XX_val=X_val, yy=y_train, yy_val=y_val, params=nn_init_q3, alpha=alpha_opt)
#     print('The above RMSE is a new baseline.')
#     # alpha_set = np.append(alpha_set, alpha_acquisition[np.argmax(PI)])

In [31]:
train_nn_reg(XX=X_train, XX_val=X_val, yy=y_train, yy_val=y_val, params=nn_init_q3, 
             alpha=best_alpha, report_train=True, report_val=True)

The RMSE_train is 0.0658 while alpha is 0.32.
The RMSE_val is 0.2436 while alpha is 0.32.


0.24361278595360691